<a href="https://colab.research.google.com/github/mapmatters/review_insight/blob/main/1nd_trial_using_sample_set_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/master_degree/paper')
!ls

Mounted at /content/drive
 논문계획서_2nd.gdoc			  data
 archive.zip				 '기존 분석결과.gdoc'
 booking.com_hotel_review_korea.csv.zip  '논문 지도교수 배정신청서.gdoc'
 code_R					 '레퍼런스 링크.gdoc'
 colab


In [2]:
import pandas as pd
import pickle
import json
from io import StringIO
import sys

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [155]:
# stopwords 점검
original_stops = stopwords.words('english')
print(len(original_stops))
my_stops = ['zoom', 'fbm'] + original_stops
print(my_stops)

179
['zoom', 'fbm', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sam

In [21]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in my_stops] # apply customized stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english'] 
    return keywords

def build_freqs(text):
    freqs = {}
    for word in preprocess_text(text):
        if word in freqs:
            freqs[word] += 1
        else:
            freqs[word] = 1
    return freqs

# >>> l = ["a","b","b"]
# >>> [[x,l.count(x)] for x in set(l)]
# [['a', 1], ['b', 2]]
# >>> dict((x,l.count(x)) for x in set(l))
# {'a': 1, 'b': 2}


In [99]:
df = pd.read_pickle('data/processed/rv_en_1.pkl')

In [102]:
print(df.info())
df.reset_index(inplace=True, drop=True)
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250624 entries, 0 to 1250623
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   app_nm   1250624 non-null  object 
 1   review   1250624 non-null  object 
 2   rating   1250624 non-null  float64
 3   dt       1250624 non-null  object 
 4   app_ver  1250624 non-null  object 
 5   lang     1250624 non-null  object 
dtypes: float64(1), object(5)
memory usage: 57.2+ MB
None


,app_nm,review,rating,dt,app_ver,lang
1250614,fbm,Cool way to communicate,4.0,2020-12-23,294.0.0.20.129,en
1250615,fbm,Good,3.0,2020-12-26,288.0.0.15.118,en
1250616,fbm,It's the worst ever it hangs and as i reboot m...,5.0,2020-12-24,257.1.0.21.120,en
1250617,fbm,No se puede dibujar sobre los las esquinas el ...,5.0,2020-12-27,294.0.0.20.129,en
1250618,fbm,You can't see pictures and videos you have sen...,3.0,2020-12-24,294.0.0.24.129,en
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en
1250622,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en
1250623,fbm,Censor this Facebook!!!🖕🖕🖕🖕🖕,1.0,2020-12-23,283.0.0.16.120,en


# Dataset describe

In [103]:
print(df['app_nm'].unique())
print('Number of apps: ',len(df['app_nm'].unique()))

['zoom' 'discord' 'fbm']
Number of apps:  3


In [104]:
print('rows and columns', df.shape, '\n')
print('reviews of each app', df.groupby(['app_nm']).size())
print('reviews of each app', df.groupby(['app_nm']).count())

rows and columns (1250624, 6) 

reviews of each app app_nm
discord    118292
fbm        795610
zoom       336722
dtype: int64
reviews of each app          review  rating      dt  app_ver    lang
app_nm                                          
discord  118292  118292  118292   118292  118292
fbm      795610  795610  795610   795610  795610
zoom     336722  336722  336722   336722  336722


# WHAT TO DO?
### preprocessing

reference 1: https://towardsdatascience.com/introduction-to-nlp-part-1-preprocessing-text-in-python-8f007d44ca96

reference 2: https://bkshin.tistory.com/entry/NLP-4-%EC%96%B4%EA%B0%84-%EC%B6%94%EC%B6%9CStemming%EA%B3%BC-%ED%91%9C%EC%A0%9C%EC%96%B4-%EC%B6%94%EC%B6%9CLemmatization

* tokenize
* lemmatize
* build frequencies

In [105]:
# 리뷰 텍스트만 list 로 추출
rv = list(df['review'])
print(len(rv))

1250624


In [107]:
# 키워드별 count (dict 형태)
rv_dict = dict()
for i in range(len(rv)):
  if i%10000==0:
    print(i, end=" - ")
  if i%100000==0:
    print('\n')
  rv_dict[i] = build_freqs(rv[i])

0 - 

10000 - 20000 - 30000 - 40000 - 50000 - 60000 - 70000 - 80000 - 90000 - 100000 - 

110000 - 120000 - 130000 - 140000 - 150000 - 160000 - 170000 - 180000 - 190000 - 200000 - 

210000 - 220000 - 230000 - 240000 - 250000 - 260000 - 270000 - 280000 - 290000 - 300000 - 

310000 - 320000 - 330000 - 340000 - 350000 - 360000 - 370000 - 380000 - 390000 - 400000 - 

410000 - 420000 - 430000 - 440000 - 450000 - 460000 - 470000 - 480000 - 490000 - 500000 - 

510000 - 520000 - 530000 - 540000 - 550000 - 560000 - 570000 - 580000 - 590000 - 600000 - 

610000 - 620000 - 630000 - 640000 - 650000 - 660000 - 670000 - 680000 - 690000 - 700000 - 

710000 - 720000 - 730000 - 740000 - 750000 - 760000 - 770000 - 780000 - 790000 - 800000 - 

810000 - 820000 - 830000 - 840000 - 850000 - 860000 - 870000 - 880000 - 890000 - 900000 - 

910000 - 920000 - 930000 - 940000 - 950000 - 960000 - 970000 - 980000 - 990000 - 1000000 - 

1010000 - 1020000 - 1030000 - 1040000 - 1050000 - 1060000 - 1070000 - 1080000 - 10

In [109]:
# word 추출 (list)
keywords_l = []
for i in range(len(rv)):
  a = list(rv_dict[i].keys())
  keywords_l.append(a)

In [110]:
# 역토큰화 (list)
detoken_1 = []
for i in range(len(rv)):
  a = ' '.join(keywords_l[i])
  detoken_1.append(a)

In [138]:
# key value (list)
items_l = []
for i in range(len(rv)):
  a = rv_dict[i]
  items_l.append(a)

In [139]:
df2 = pd.DataFrame.from_dict({'keywords':keywords_l, 
                              'detoken':detoken_1,
                              'items': items_l})

In [143]:
df_m = pd.merge(df, df2, left_index=True, right_index=True)

In [144]:
df_m

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}"
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':..."
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1}
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}"
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1}
...,...,...,...,...,...,...,...,...,...
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en,"[problem, archive, story, many, see]",problem archive story many see,"{'problem': 1, 'archive': 1, 'story': 2, 'many..."
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en,"[way, change, individual, notification, sound,...",way change individual notification sound whole...,"{'way': 1, 'change': 2, 'individual': 1, 'noti..."
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en,"[latest, update, ruin, app, chat, bubble, suck...",latest update ruin app chat bubble suck notifi...,"{'latest': 1, 'update': 1, 'ruin': 1, 'app': 1..."
1250622,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en,"[since, android, 11, app, baaaad, update, noti...",since android 11 app baaaad update notice chat...,"{'since': 1, 'android': 1, '11': 1, 'app': 1, ..."


In [145]:
df_m.to_pickle('data/processed/rv_en_2.pkl')

# TF-IDF, Topic modeling

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
df = pd.read_pickle('data/processed/rv_en_2.pkl')
df.head(10)

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}"
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':..."
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1}
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}"
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1}
5,zoom,Pro,5.0,2020-12-26,5.2.42588.0803,en,[pro],pro,{'pro': 1}
6,zoom,It's very very useful for school students,5.0,2020-12-26,5.4.7.946,en,"[useful, school, students]",useful school students,"{'useful': 1, 'school': 1, 'students': 1}"
7,zoom,Max 4 people on one screen including myself. T...,1.0,2020-12-23,Unknown,en,"[max, 4, people, one, screen, include, usable]",max 4 people one screen include usable,"{'max': 1, '4': 1, 'people': 1, 'one': 1, 'scr..."
8,zoom,Super,5.0,2020-12-23,5.4.4.615,en,[super],super,{'super': 1}
9,zoom,Won't let me Unmute. . The level of Simplicity...,3.0,2020-12-24,Unknown,en,"[win, let, unmute, level, simplicity, complicate]",win let unmute level simplicity complicate,"{'win': 1, 'let': 1, 'unmute': 1, 'level': 1, ..."


In [12]:
# len(df['keywords'][0])
df['n_words'] = df['keywords'].apply(lambda x: len(x))
df

,app_nm,review,rating,dt,app_ver,lang,keywords,detoken,items,n_words
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en,"[ভ, ল]",ভ ল,"{'ভ': 1, 'ল': 1}",2
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en,"[one, education, thing, cant, unlike, never]",one education thing cant unlike never,"{'one': 1, 'education': 1, 'thing': 1, 'cant':...",6
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en,[good],good,{'good': 1},1
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en,"[worst, appp]",worst appp,"{'worst': 1, 'appp': 1}",2
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en,[good],good,{'good': 1},1
...,...,...,...,...,...,...,...,...,...,...
1250619,fbm,problem in archive story and how many other se...,1.0,2020-12-25,290.0.0.16.119,en,"[problem, archive, story, many, see]",problem archive story many see,"{'problem': 1, 'archive': 1, 'story': 2, 'many...",5
1250620,fbm,why isn't there a way to change an individual'...,1.0,2020-12-28,294.0.0.24.129,en,"[way, change, individual, notification, sound,...",way change individual notification sound whole...,"{'way': 1, 'change': 2, 'individual': 1, 'noti...",24
1250621,fbm,The latest update ruined this app... The chat ...,1.0,2020-12-28,294.0.0.24.129,en,"[latest, update, ruin, app, chat, bubble, suck...",latest update ruin app chat bubble suck notifi...,"{'latest': 1, 'update': 1, 'ruin': 1, 'app': 1...",14
1250622,fbm,Since Android 11 the app is baaaad. That or it...,2.0,2020-12-26,294.0.0.24.129,en,"[since, android, 11, app, baaaad, update, noti...",since android 11 app baaaad update notice chat...,"{'since': 1, 'android': 1, '11': 1, 'app': 1, ...",16


In [28]:
# df[['n_words']].groupby(['n_words']).size()
df_model = df[df['n_words']>4]
print("\ngroup by n_words", df_model.groupby(['n_words']).size())
print("\ngroup by rating", df_model.groupby(['rating']).size())
print(df_model.shape)

df_model_1 = df_model[df_model['rating']==1.0]
df_model_5 = df_model[df_model['rating']==5.0]
print("\nrating 1: ", df_model_1.shape)
print("\nrating 5: ", df_model_5.shape)


group by n_words n_words
5      54902
6      42271
7      33990
8      27470
9      23093
       ...  
96         1
105        1
108        1
109        1
127        1
Length: 88, dtype: int64

group by rating rating
1.0    115922
2.0     28272
3.0     32997
4.0     36762
5.0    107568
dtype: int64
(321521, 10)

rating 1:  (115922, 10)

rating 5:  (107568, 10)


In [29]:
# rating = 5
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(df_model_5['detoken'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# LDA
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

lda_top = lda_model.fit_transform(X)
# print(lda_model.components_)
print(lda_model.components_.shape) 

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])
        # print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

(107568, 1000)
[[2.26372770e+01 1.00008587e-01 1.00002510e-01 ... 1.00000094e-01
  1.00000012e-01 1.00000164e-01]
 [1.00020082e-01 1.00017239e-01 1.00008304e-01 ... 1.00000560e-01
  1.00000136e-01 1.00000300e-01]
 [1.00023074e-01 1.00013503e-01 1.00009696e-01 ... 1.00000008e-01
  1.00000014e-01 1.00000327e-01]
 ...
 [1.00033812e-01 1.00034169e-01 1.00009648e-01 ... 1.00002216e-01
  1.97189141e+02 1.00002902e-01]
 [1.88448206e+02 1.00031015e-01 6.88037830e+01 ... 1.00000205e-01
  1.00001044e-01 1.00001047e-01]
 [1.00005856e-01 1.00008386e-01 1.00004022e-01 ... 1.00000466e-01
  1.00000016e-01 1.00000254e-01]]
(10, 1000)
Topic 1: ['family', 'friends', 'communicate', 'love', 'way', 'talk', 'contact', 'connect', 'touch', 'great']
Topic 2: ['messenger', 'facebook', 'text', 'video', 'message', 'app', 'fix', 'problem', 'good', 'send']
Topic 3: ['na', 'awesome', 'know', 'let', 'data', 'person', 'star', 'number', 'messager', 'head']
Topic 4: ['communication', 'work', 'ones', 'think', 'game', 'fb

In [30]:
# rating = 1
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(df_model_1['detoken'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# LDA
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape) 

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])
        # print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

(115922, 1000)
[[1.00024816e-01 1.00008713e-01 2.92288283e+02 ... 5.31382960e+00
  1.00008712e-01 1.00001035e-01]
 [3.38074725e-01 1.00008608e-01 1.00011983e-01 ... 5.50357930e-01
  1.33183944e+02 1.00000702e-01]
 [1.00021643e-01 1.00016802e-01 1.00034772e-01 ... 8.86730461e-01
  1.00008842e-01 1.00001127e-01]
 ...
 [1.00022000e-01 1.00008763e-01 1.00053601e-01 ... 6.64846834e+00
  1.00006786e-01 1.00000941e-01]
 [1.00011436e-01 1.00009741e-01 1.00015298e-01 ... 9.66288661e-01
  1.12136873e-01 1.00000017e-01]
 [1.13528485e+01 1.01708368e+02 1.00013223e-01 ... 4.72824046e+00
  1.00004994e-01 1.00000834e-01]]
(10, 1000)
Topic 1: ['update', 'message', 'friends', 'fix', 'send', 'latest', 'videos', 'know', 'version', 'new']
Topic 2: ['message', 'send', 'bug', 'slow', 'lag', 'fix', 'wait', 'photo', 'appear', 'update']
Topic 3: ['photos', 'bubble', 'chat', 'conversation', 'wala', 'app', 'message', 'girls', 'install', 'great']
Topic 4: ['receive', 'send', 'active', 'update', 'reinstall', 'try'

#### Gensim 으로 동일한 과정 해보기

In [31]:
from gensim import corpora

In [32]:
tokenized_doc = df_model_5['keywords']
tokenized_doc

1               [one, education, thing, cant, unlike, never]
30         [good, app, much, net, problem, poor, video, q...
56         [2020, way, life, connect, friends, family, work]
68                [nice, every, new, update, face, surprise]
72         [zoom, awesome, recommend, app, meet, face, ea...
                                 ...                        
1250588    [helpful, maintain, contact, relatives, friend...
1250590            [𝙼𝚊𝚐𝚊𝚗𝚍𝚊, 𝚔𝚊𝚜𝚘, 𝚕𝚗𝚐, 𝚖𝚒𝚗𝚜𝚊𝚗, 𝚗𝚊𝚐, 𝚕𝚘𝚕𝚘𝚔𝚘]
1250591                         [love, app, favorite, 1, go]
1250616    [worst, ever, hang, reboot, cell, disappear, r...
1250617    [se, puede, dibujar, sobre, los, las, esquinas...
Name: keywords, Length: 107568, dtype: object

In [64]:
dictionary = corpora.Dictionary(tokenized_doc)
print(dictionary[36])
print(len(dictionary))

thank
63127


In [65]:
from collections import Counter

min_count = 50
word_counter = Counter((word for words in tokenized_doc for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}

dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary)) # dictionary size : 10354

dictionary size : 1710


In [66]:
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 리뷰 출력. 첫번째 문서의 인덱스는 0
corpus

[(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)],
 [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)],
 [(6, 1), (22, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)],
 [(29, 1), (33, 1), (34, 1), (35, 1)],
 [(6, 1), (34, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)],
 [(41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1)],
 [(45, 1),
  (48, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(6, 1),
  (13, 1),
  (24, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1)],
 [(6, 1), (39, 1), (67, 1), (68, 1), (69, 1)],
 [(6, 1),
  (11, 1),
  (29, 1),
  (31, 1),
  (32, 1),
  (38, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1)],
 [(7, 1),
  (13, 1),
  (74, 1),
 

In [69]:
import gensim
from gensim.models import LdaModel
NUM_TOPICS = 10 #20개의 토픽, k=20
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)

FileNotFoundError: ignored

In [74]:
topics = lda_model.print_topics(num_words=10)

In [75]:
for topic in topics:
    print(topic)
print(lda_model.print_topics())

(0, '0.092*"app" + 0.089*"best" + 0.043*"apps" + 0.035*"friend" + 0.035*"download" + 0.030*"ever" + 0.029*"one" + 0.026*"chat" + 0.025*"love" + 0.023*"use"')
(1, '0.052*"ko" + 0.051*"na" + 0.034*"ka" + 0.031*"wala" + 0.026*"load" + 0.022*"app" + 0.019*"ok" + 0.017*"hai" + 0.017*"log" + 0.015*"kachat"')
(2, '0.040*"friends" + 0.037*"message" + 0.034*"love" + 0.029*"family" + 0.029*"keep" + 0.026*"great" + 0.025*"app" + 0.021*"people" + 0.020*"get" + 0.019*"way"')
(3, '0.040*"facebook" + 0.028*"update" + 0.022*"u" + 0.021*"get" + 0.020*"back" + 0.018*"new" + 0.017*"send" + 0.017*"go" + 0.017*"ng" + 0.015*"know"')
(4, '0.099*"call" + 0.097*"chat" + 0.085*"video" + 0.050*"app" + 0.037*"good" + 0.026*"message" + 0.020*"problem" + 0.019*"fix" + 0.019*"please" + 0.016*"share"')
(5, '0.076*"friends" + 0.070*"app" + 0.059*"use" + 0.049*"family" + 0.048*"love" + 0.048*"easy" + 0.032*"talk" + 0.029*"good" + 0.027*"communicate" + 0.026*"great"')
(6, '0.120*"app" + 0.057*"good" + 0.055*"thank" + 0.

In [70]:
with open('data/model/', 'wb') as f:
    pickle.dump(lda_model, f)

IsADirectoryError: ignored

### LDA 시각화

In [ ]:
import pyLDAvis.gensim